In [ ]:
# %load miniflow.py
import numpy as np

class Node(object):

    def __init__(self, inbound_nodes=[]):
        
        self.inbound_nodes = inbound_nodes

        self.outbound_nodes = []        

        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)

        self.value = None

    def forward(self):
        raise NotImplemented

class Input(Node):

    def __init__(self):
        Node.__init__(self)

    def forward(self, value=None):
        
        if value is not None:
            self.value = value

class Add(Node):

    def __init__(self, x, y):
        Node.__init__(self, [x, y])

    def forward(self):
        self.value = 0
        for i in range(len(self.inbound_nodes)):
            self.value += self.inbound_nodes[i].value
        return self.value 

class Sigmoid(Node):

    def __init__(self, n):
        Node.__init__(self, [n])

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def forward(self):
        self.value = self._sigmoid(self.inbound_nodes[0].value) 
        return self.value
    
class Linear(Node):

    def __init__(self, inputs, weights, bias):
        Node.__init__(self, [inputs, weights, bias])

    def forward(self):
        inputs = self.inbound_nodes[0].value
        weights = self.inbound_nodes[1].value
        bias = self.inbound_nodes[2].value
        self.value = sum(map(lambda x,y:x*y, inputs,weights)) + bias 
        return self.value
    
class LinearMatrix(Node):

    def __init__(self, X, W, B):
        Node.__init__(self, [X, W, B])

    def forward(self):

        X = self.inbound_nodes[0].value
        W = self.inbound_nodes[1].value
        B = self.inbound_nodes[2].value

        self.value = X.dot(W) + B 
        return self.value

class MSE(Node):

    def __init__(self, y, a):
        Node.__init__(self, [y, a])

    def forward(self):

        y = self.inbound_nodes[0].value.reshape(-1,1)
        a = self.inbound_nodes[1].value.reshape(-1,1)

        #self.value =  sum(np.square(y - a)) / len(y)
        self.value = np.mean(np.square(y - a))

'''
Kahn's algorihtm
'''
def topological_sort(feed_dict):
    
    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        G.setdefault(n, {'in':set(), 'out': set()})

        for it in n.outbound_nodes:
            G.setdefault(it, {'in':set(), 'out':set()})
            G[n]['out'].add(it)
            G[it]['in'].add(n)

            nodes.append(it)

    L = []
    S = set(input_nodes)
    while (len(S) > 0):
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            if  len(G[m]['in']) == 0:
                S.add(m)
    #if G:
        #raise RuntimeError("graph has at least one cycle")
        
    return L

def forward_pass(output_node, sorted_nodes):

    for n in sorted_nodes:
        n.forward()

    return output_node.value

def forward_pass_no_ret(sorted_nodes):

    for n in sorted_nodes:
        n.forward()



In [ ]:
%run miniflow.py

x, y = Input(), Input()

f = Add(x, y)

feed_dict = {x:10, y:5}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)
print("{} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))


In [ ]:
%run miniflow.py


import numpy as np


X, W, b = Input(), Input(), Input()

f = LinearMatrix(X, W, b)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

"""
Output should be:
[[-9., 4.],
[-9., 4.]]
"""
print(output)

In [ ]:
%run miniflow.py

import numpy as np


X, W, b = Input(), Input(), Input()

f = LinearMatrix(X, W, b)
g = Sigmoid(f)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(g, graph)

"""
Output should be:
[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]
"""
print(output)

In [ ]:
%run miniflow.py
import numpy as np


y, a = Input(), Input()
cost = MSE(y, a)

y_ = np.array([1, 2, 3])
a_ = np.array([4.5, 5, 10])

feed_dict = {y: y_, a: a_}
graph = topological_sort(feed_dict)
# forward pass
forward_pass_no_ret(graph)

"""
Expected output

23.4166666667
"""
print(cost.value)

In [ ]:
%run f.py